<a href="https://colab.research.google.com/github/amanjain1397/zip-the-code-1.0/blob/master/v_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

!mkdir -p drive
!google-drive-ocamlfuse drive

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131323 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [2]:
pwd

'/content'

In [3]:
cd drive/DL/Keras - Text

/content/drive/DL/Keras - Text


In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import re
import keras

import nltk 
nltk.download('punkt')

nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))

In [0]:
def preprocess_text(text):
    
    #Lowering case the text
    text = text.lower()
    
    #Removing numbers
    text = re.sub(r'\d+', '', text)
    
    #Removing punctuations
    import string
    from nltk.tokenize import word_tokenize
    text = text.translate(str.maketrans('', '', string.punctuation))

    #Removing stop words
    
    tokens = word_tokenize(text)
    text = ' '.join([ i for i in tokens if not i in stopwords])
    
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    return text

In [8]:
data = pd.read_csv('./data/train_file.csv')
test = pd.read_csv('./data/test_file.csv')
sample = pd.read_csv('./data/sample_submission.csv')

data.loc[data.Subjects.isnull(), 'Subjects'] = ''
test.loc[test.Subjects.isnull(), 'Subjects'] = ''

data['combined'] = data.Subjects.str.cat(' ' + data.Title)
test['combined'] = test.Subjects.str.cat(' ' + test.Title)

dataMatType = data.MaterialType
data.drop(labels = ['MaterialType'], axis = 1, inplace = True)

data.combined = data.combined.apply(preprocess_text)
test.combined = test.combined.apply(preprocess_text)
'''
data.Subjects = data.Subjects.apply(preprocess_text)
data.Title = data.Title.apply(preprocess_text)
test.Subjects = test.Subjects.apply(preprocess_text)
test.Title = test.Title.apply(preprocess_text)
'''
flag = -1
labelDict = {}

for key in dataMatType.value_counts().keys():
  flag += 1
  labelDict[key] = flag
  
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

maxlen = 10                             

max_words = 1000                                    

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data.combined)

dataSequences = tokenizer.texts_to_sequences(data.combined)
testSequences = tokenizer.texts_to_sequences(test.combined)

data_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(data_word_index))

Found 36088 unique tokens.


In [0]:
from keras import preprocessing
x_train = preprocessing.sequence.pad_sequences(dataSequences, maxlen = 10)
x_test = preprocessing.sequence.pad_sequences(testSequences, maxlen = 10)

train_label = dataMatType.map(labelDict)

from keras.utils import to_categorical
labelDataBinary = to_categorical(train_label)

from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(dataMatType), dataMatType)

In [10]:
import os

embeddings_index = {}
f = open( './glove.6B/glove.6B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

embedding_dim = 300

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in data_word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

Found 400000 word vectors.


In [78]:
from keras.layers import LSTM
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding, SimpleRNN, Conv1D, MaxPooling1D
from keras.layers import Dropout
from keras.layers import Dense

model = Sequential()
model.add(Embedding(1000, 300, input_length=10))

'''model.add(Conv1D(64, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=4))
'''


model.add(Conv1D(128, 3,  activation='relu'))
model.add(Dropout(0.3))

model.add(LSTM(400, recurrent_dropout = 0.6, return_sequences = True))
model.add(Dropout(0.6))

model.add(LSTM(400, recurrent_dropout = 0.2))
model.add(Dropout(0.2))

model.add(Dense(8, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])
history = model.fit(x_train, labelDataBinary,
                    epochs=8,
                    validation_split=0.2,
                    class_weight = class_weights)

model.save('./models/m5.h5')

Train on 25322 samples, validate on 6331 samples
Epoch 1/8
25322/25322 [==============================] - 58s 2ms/step - loss: 0.5827 - acc: 0.8373 - val_loss: 0.4636 - val_acc: 0.8727
Epoch 2/8
25322/25322 [==============================] - 50s 2ms/step - loss: 0.4632 - acc: 0.8698 - val_loss: 0.4404 - val_acc: 0.8785
Epoch 3/8
25322/25322 [==============================] - 50s 2ms/step - loss: 0.4425 - acc: 0.8743 - val_loss: 0.4657 - val_acc: 0.8789
Epoch 4/8
25322/25322 [==============================] - 49s 2ms/step - loss: 0.4271 - acc: 0.8776 - val_loss: 0.4289 - val_acc: 0.8807
Epoch 5/8
25322/25322 [==============================] - 49s 2ms/step - loss: 0.4190 - acc: 0.8805 - val_loss: 0.4479 - val_acc: 0.8782
Epoch 6/8
25322/25322 [==============================] - 49s 2ms/step - loss: 0.4146 - acc: 0.8804 - val_loss: 0.4816 - val_acc: 0.8837
Epoch 7/8
25322/25322 [==============================] - 49s 2ms/step - loss: 0.4115 - acc: 0.8822 - val_loss: 0.4838 - val_acc: 0.8833

In [0]:
model.save('./models/m1.h5')

**Using Embedding Layer**

In [0]:
from keras.layers import LSTM
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding, SimpleRNN, Conv1D, MaxPooling1D
from keras.layers import Dropout
from keras.layers import Dense

model = Sequential()
model.add(Embedding(1000, 300, input_length=10, weights=[embedding_matrix], trainable=False))

'''model.add(Conv1D(64, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=4))
'''

model.add(LSTM(1000, recurrent_dropout = 0.3))
model.add(Dropout(0.35))
model.add(Dense(8, activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['acc'])
history = model.fit(x_train, labelDataBinary,
                    epochs=8,
                    batch_size=128,
                    validation_split=0.2,
                    class_weight = class_weights)

In [0]:
reverseLabelDict = {}

for key, value in labelDict.items():
  reverseLabelDict[value] = key
  
predictions = model.predict(x_test)
predictions = pd.Series([np.argmax(i) for i in predictions], index = test.index)
predictions = predictions.map(reverseLabelDict)

submission = pd.concat([test.ID.astype(np.int), predictions], axis = 1)
submission.columns = ['ID' , 'MaterialType']

In [0]:
submission.to_csv('submit.csv', index = False, header = True)